# Annotating plots - Gaia - CMD

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Rectangle, Circle, Ellipse, Polygon

from astropy.table import QTable
from astropy import units as u
from astroquery.gaia import Gaia

### NGC 2682 (M 67) is a very well studied open star cluster in the northern skies

&nbsp;

<p>
<img src="https://uwashington-astro300.github.io/A300_images/M67.jpg" width = "500">
</p>


- Right ascension: 08h 51.3m (132.825 deg)
- Declination: +11° 49′ (11.817 deg)

In [ ]:
my_object_ra = 132.825
my_object_dec = 11.817

---

## Gaia query for 10σ data for our target

- Note the use of `f-string` and variables in the query
- Gaia wants the object coordinates in degrees

In [ ]:
my_query = f"""
SELECT TOP 1500
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax, parallax_over_error
FROM gaiadr3.gaia_source_lite
WHERE DISTANCE( POINT({my_object_ra}, {my_object_dec}), POINT(ra, dec) ) < 0.40
AND parallax_over_error > 10
AND bp_rp IS NOT NULL
ORDER BY parallax DESC
"""

In [ ]:
print(my_query)

### Since we are running into time-out issues with the Gaia server, you should use `Gaia.launch_job_async(my_query)` for your queries. 

In [ ]:
my_job_query = Gaia.launch_job_async(my_query)

In [ ]:
print(my_job_query)

### Query returned 1399 objects. Less than the `SELECT` limit of 1500, so I am not cutting off objects

In [ ]:
my_table = my_job_query.get_results()

In [ ]:
#my_table = QTable.read('./Cluster_Table.ecsv', format='ascii.ecsv')

In [ ]:
my_table[0:2]

### Use `parallax` and `phot_g_mean_mag` to get **distance** and **absolute magnitude**

- Add then as columns to the data table

In [ ]:
my_table['distance'] = my_table['parallax'].to(u.parsec, equivalencies=u.parallax())

In [ ]:
my_table[0:2]

In [ ]:
def find_absmag(my_gmag, my_distance):
    result = my_gmag - 5 * np.log10( my_distance / (10 * u.parsec)) * u.mag
    return result

In [ ]:
my_table['abs_g'] = find_absmag(my_gmag = my_table['phot_g_mean_mag'], 
                                 my_distance = my_table['distance']
                               ) * u.mag

In [ ]:
my_table[0:2]

## Plot a histogram of the distances

- All the stars in the cluster should have the same distance
- Objects not in the cluster will have different distances

In [ ]:
fig, ax = plt.subplot_mosaic(
    '''
    AB
    ''',
    figsize = (12, 4), 
    constrained_layout = True
)

ax['A'].set_xlabel("Distance (pc)")
ax['A'].set_ylabel("Number")

ax['A'].hist(my_table['distance'],
        bins = 100,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid')

ax['B'].set_xlim(500, 1250)

ax['B'].set_xlabel("Distance (pc)")
ax['B'].set_ylabel("Number")

ax['B'].hist(my_table['distance'],
        bins = 100,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid');

## Pretty easy to see the cluster's distance

- We will use distances between 750 pc and 1000 pc for cluster stars
- The cluster distance from the literature is 800 - 900 pc, so these values seems fine.

In [ ]:
my_cluster_table = my_table[(my_table['distance'] > 750) &
                            (my_table['distance'] < 1000)
                           ]

In [ ]:
np.size(my_cluster_table)

In [ ]:
np.size(my_cluster_table) / np.size(my_table)

---

## Side Topic - Combining Tables

If you have two tables that have the exact same columns, you can combine them with the `vstack` command

In [ ]:
from astropy.table import vstack

In [ ]:
my_small_table_one = my_table[(my_table['distance'] > 750) &
                              (my_table['distance'] <= 850)
                             ]

In [ ]:
np.size(my_small_table_one)

In [ ]:
my_small_table_two = my_table[(my_table['distance'] > 850) &
                              (my_table['distance'] < 1000)
                             ]

In [ ]:
np.size(my_small_table_two)

In [ ]:
my_combined_table = vstack(
    [my_small_table_one, my_small_table_two]
)

In [ ]:
np.size(my_combined_table)

---

## Make a color magnitude diagram (CMD) of the object


Color Magnitude Diagram (CMD) is a plot of Color Index vs. Magnitude. This is just a HR-diagram with a change of units.

Some things we have to keep in mind when making a CMD

- Color Index (X-axis) cover a very small range of values, we will need to adjust our axes accordingly.
- Magnitudes (Y-axis) are backwards, we will need to adjust our axes accordingly.

#### `np.ptp()` returns the range of values (max - min) for an array (*P*eak *T*o *P*eak)

In [ ]:
np.ptp(my_cluster_table['bp_rp'])

In [ ]:
np.ptp(my_cluster_table['abs_g'])

#### You can see the range of the x-axis is small compared to the range of the Y-axis. This can lead to long narrow plot.

- We can use the `ax.set_aspect(aspect_ratio)` command to deal with this.
- Where the `aspect_ratio` what you need to multiply the Y range by to get the X range.
- In this case 9.8 / 5 ~ 2, so `aspect_ratio` = 1/5

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

ax.set_aspect(1 / 5)

# Magnitudes are backwards

ax.set_ylim(-2.5,10)
ax.invert_yaxis()

ax.tick_params(axis='both', which='major', labelsize = 18)

###

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel("G_Mag",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title("M67 CMD",
             fontfamily = 'serif',
             fontsize = 30)

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5
       );